<a href="https://colab.research.google.com/github/sanchesprogramer/Inteligencia-Artificial/blob/main/Projeto/An%C3%A1lise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Raphael Dias Sanches RA: 10403418

Felipe de Almeida Parreira RA: 10400771

In [ ]:
#!pip install xgboost
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_absolute_error

# Modelos de regressão
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge

In [ ]:
dataset = pd.read_csv("https://raw.githubusercontent.com/sanchesprogramer/Inteligencia-Artificial/refs/heads/main/Projeto/Base.csv", sep = ";")
dataset_rio_pardo = pd.read_csv("https://raw.githubusercontent.com/sanchesprogramer/Inteligencia-Artificial/refs/heads/main/Projeto/INMET_S_RS_A813_RIO PARDO_01-01-2024_A_31-12-2024.CSV", sep = ";")
dataset_inicio_inundacao = pd.read_csv("https://raw.githubusercontent.com/sanchesprogramer/Inteligencia-Artificial/refs/heads/main/Projeto/Mooca123.csv", sep = ";")

In [ ]:
dataset_rio_pardo.head()

In [ ]:
dataset = dataset.drop(columns=['RADIACAO GLOBAL (Kj/m²)']) #Não será levado em consideração a radiação global nesse projeto
colunas_numericas = dataset.columns.difference(["Data", "Hora UTC"])
dataset[colunas_numericas] = dataset[colunas_numericas].astype(str).replace(",", ".", regex=True).astype(float)
for col in dataset.columns:
    if dataset[col].dtype == 'object':
        try:
            dataset[col] = pd.to_datetime(dataset[col])
        except:
            pass

In [ ]:
dataset = dataset.dropna()
dataset = dataset.reset_index(drop=True)
dataset.head()

,Data,Hora UTC,"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)"
0,01/01/2024,0000 UTC,0.0,1010.6,1010.2,19.6,14.4,20.5,19.6,14.4,13.8,72.0,65.0,72.0,123.0,6.5,1.5
1,01/01/2024,0100 UTC,0.0,1011.2,1010.6,19.0,14.4,19.6,19.0,14.4,14.0,75.0,70.0,75.0,137.0,5.3,2.1
2,01/01/2024,0200 UTC,0.0,1011.3,1011.1,18.6,14.6,19.0,18.6,14.7,14.4,78.0,75.0,78.0,121.0,4.6,1.9
3,01/01/2024,0300 UTC,0.0,1011.1,1010.4,18.8,14.9,18.9,18.6,14.9,14.6,79.0,77.0,78.0,119.0,5.0,1.6
4,01/01/2024,0400 UTC,0.0,1010.5,1010.1,18.9,14.9,19.2,18.8,15.0,14.7,78.0,75.0,77.0,114.0,5.2,1.5


In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8746 entries, 0 to 8745
Data columns (total 17 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   Data                                              8746 non-null   object 
 1   Hora UTC                                          8746 non-null   object 
 2   PRECIPITAÇÃO TOTAL, HORÁRIO (mm)                  8746 non-null   float64
 3   PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)   8746 non-null   float64
 4   PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)  8746 non-null   float64
 5   TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)      8746 non-null   float64
 6   TEMPERATURA DO PONTO DE ORVALHO (°C)              8746 non-null   float64
 7   TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)        8746 non-null   float64
 8   TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)        8746 non-null   float64
 9   TEMPERATURA ORVALHO

In [ ]:
dataset.describe()

,"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)"
count,8746.000000,8746.000000,8746.000000,8746.000000,8746.000000,8746.000000,8746.000000,8746.000000,8746.000000,8746.000000,8746.000000,8746.000000,8746.000000,8746.000000,8746.000000
mean,0.238052,1009.847519,1009.256506,20.350960,16.050343,20.870947,19.870695,16.536611,15.617962,80.718843,75.431740,78.116396,153.747656,4.924354,1.464864
std,1.469494,5.295553,5.375816,5.603711,4.655168,5.758879,5.459374,4.645837,4.692184,13.487006,15.810303,14.786143,80.106005,2.572680,0.780496
min,0.000000,990.100000,988.900000,3.400000,-0.700000,3.500000,3.100000,0.100000,-1.800000,33.000000,26.000000,28.000000,1.000000,0.600000,0.100000
25%,0.000000,1006.300000,1005.700000,17.000000,13.500000,17.300000,16.600000,14.000000,13.000000,72.000000,64.000000,68.000000,103.000000,3.000000,0.900000
50%,0.000000,1009.400000,1008.800000,20.500000,16.800000,20.900000,20.100000,17.200000,16.300000,84.000000,79.000000,82.000000,129.000000,4.600000,1.300000
75%,0.000000,1013.000000,1012.500000,24.100000,19.400000,24.700000,23.600000,19.900000,19.000000,92.000000,89.000000,90.000000,193.000000,6.500000,1.900000
max,46.800000,1027.100000,1026.900000,36.700000,25.700000,37.300000,36.000000,25.800000,24.800000,97.000000,97.000000,97.000000,360.000000,25.000000,7.900000


In [ ]:
dataset.isnull().sum()

,0
Data,0
Hora UTC,0
"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)",0
PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),0
PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),0
"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",0
TEMPERATURA DO PONTO DE ORVALHO (°C),0
TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),0
TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),0
TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),0


In [ ]:
dataset['Data'] = pd.to_datetime(dataset['Data'], format='%d/%m/%Y')
dataset_filtrado = dataset[(dataset['Data'] >= "2024-04-01") & (dataset['Data'] <= "2024-05-31")]
dataset_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1446 entries, 2166 to 3611
Data columns (total 17 columns):
 #   Column                                            Non-Null Count  Dtype         
---  ------                                            --------------  -----         
 0   Data                                              1446 non-null   datetime64[ns]
 1   Hora UTC                                          1446 non-null   object        
 2   PRECIPITAÇÃO TOTAL, HORÁRIO (mm)                  1446 non-null   float64       
 3   PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)   1446 non-null   float64       
 4   PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)  1446 non-null   float64       
 5   TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)      1446 non-null   float64       
 6   TEMPERATURA DO PONTO DE ORVALHO (°C)              1446 non-null   float64       
 7   TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)        1446 non-null   float64       
 8   TEMPERATURA MÍNIMA NA HORA ANT

In [ ]:
X = dataset.drop("PRECIPITAÇÃO TOTAL, HORÁRIO (mm)", axis=1)
y = dataset["PRECIPITAÇÃO TOTAL, HORÁRIO (mm)"]

In [ ]:
X[f"Ano"]  = X["Data"].dt.year
X[f"Mês"] = X["Data"].dt.month
X[f"Dia"]   = X["Data"].dt.day
X.drop(columns="Data", inplace=True)
X['Hora UTC'] = pd.to_datetime(X['Hora UTC'], format='%H%M %Z')
X['hour'] = X['Hora UTC'].dt.hour
X['day'] = X['Hora UTC'].dt.day
X.drop('Hora UTC', axis=1, inplace=True)

In [ ]:
#Random Forest
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
rf_model = RandomForestRegressor(
    n_estimators=300, max_depth=None, n_jobs=-1, random_state=42)

rf_pipe = Pipeline([
    ("scaler", StandardScaler()),   # mantém consistência; árvores até dispensam
    ("model", rf_model)
])

rf_pipe.fit(X_train, y_train)
rf_pred = rf_pipe.predict(X_test)

r2_RF = f"{r2_score(y_test, rf_pred):.3f}"
mae_RF = f"{mean_absolute_error(y_test, rf_pred):.3f}"

print("Random Forest")
print(f"  R² : {r2_RF}")
print(f"  MAE: {mae_RF}")

Random Forest
  R² : 0.554
  MAE: 0.248


In [ ]:
#Extra Trees
et_model = ExtraTreesRegressor(
    n_estimators=300, max_depth=None, n_jobs=-1, random_state=42)

et_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("model", et_model)
])

et_pipe.fit(X_train, y_train)
et_pred = et_pipe.predict(X_test)

r2_ET = f"{r2_score(y_test, et_pred):.3f}"
mae_ET = f"{mean_absolute_error(y_test, et_pred):.3f}"

print("Extra Trees")
print(f"  R² : {r2_ET}")
print(f"  MAE: {mae_ET}")


Extra Trees
  R² : 0.533
  MAE: 0.250


In [ ]:
#Gradient Boosting
gb_model = GradientBoostingRegressor(
    n_estimators=200, learning_rate=0.05, random_state=42)

gb_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("model", gb_model)
])

gb_pipe.fit(X_train, y_train)
gb_pred = gb_pipe.predict(X_test)

r2_GB = f"{r2_score(y_test, gb_pred):.3f}"
mae_GB = f"{mean_absolute_error(y_test, gb_pred):.3f}"

print("Gradient Boosting")
print(f"  R² : {r2_GB}")
print(f"  MAE: {mae_GB}")


Gradient Boosting
  R² : 0.636
  MAE: 0.263


In [ ]:
#XGBoost
xgb_model = XGBRegressor(
    n_estimators=300, learning_rate=0.05,
    objective="reg:squarederror", random_state=42)

xgb_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("model", xgb_model)
])

xgb_pipe.fit(X_train, y_train)
xgb_pred = xgb_pipe.predict(X_test)

r2_XB = f"{r2_score(y_test, xgb_pred):.3f}"
mae_XB = f"{mean_absolute_error(y_test, xgb_pred):.3f}"

print("XGBoost")
print(f"  R² : {r2_XB}")
print(f"  MAE: {mae_XB}")

XGBoost
  R² : 0.474
  MAE: 0.266


In [ ]:
#Stacking
stack_model = StackingRegressor(
    estimators=[
        ("rf", rf_model),
        ("et", et_model),
        ("gb", gb_model)
    ],
    final_estimator=Ridge(),
    n_jobs=-1
)

stack_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("model", stack_model)
])

stack_pipe.fit(X_train, y_train)
stack_pred = stack_pipe.predict(X_test)

r2_SK = f"{r2_score(y_test, stack_pred):.3f}"
mae_SK = f"{mean_absolute_error(y_test, stack_pred):.3f}"

print("Stacking")
print(f"  R² : {r2_SK}")
print(f"  MAE: {mae_SK}")

Stacking
  R² : 0.577
  MAE: 0.249


In [ ]:
results = pd.DataFrame({
    "Modelo": ["Random Forest", "Extra Trees", "Gradient Boosting", "XGBoost", "Stacking"],
    "R²": [float(r2_RF), float(r2_ET), float(r2_GB), float(r2_XB), float(r2_SK)],
    "MAE": [float(mae_RF), float(mae_ET), float(mae_GB), float(mae_XB), float(mae_SK)]
})

# Ordena do melhor para o pior com base no R²
results_sorted = results.sort_values(by="R²", ascending=False).reset_index(drop=True)

print("\nComparação dos Modelos (do melhor para o pior):")
print(results_sorted)


Comparação dos Modelos (do melhor para o pior):
              Modelo     R²    MAE
0  Gradient Boosting  0.636  0.263
1           Stacking  0.577  0.249
2      Random Forest  0.554  0.248
3        Extra Trees  0.533  0.250
4            XGBoost  0.474  0.266
